In [ ]:
train_loss = []   
best_rmse = []   
val_img_loss = []
val_img_rmse = []

## Import

In [ ]:
import matplotlib.pyplot as plt
import random                                       # random 모듈 가져오기
import pandas as pd                                 # pandas 모듈 가져오기
import numpy as np                                  # numpy 모듈 가져오기
import os                                           # os 모듈 가져오기
import glob                                         # glob 모듈 가져오기
import cv2                                          # cv2 모듈 가져오기

import torch                                        # torch 모듈 가져오기
import torch.nn as nn                               # torch.nn 모듈(데이터 구조나 레이어 등이 정의) 가져오기
import torch.optim as optim                         # torch.optim(최적화 알고리즘이 구현) 가져오기
import torch.nn.functional as F                     # nn을 함수형으로 가져오기
from torch.utils.data import Dataset, DataLoader    # 데이터를 python으로 연결할 수 있도록 설정

from tqdm.auto import tqdm                          # 깔끔하게 출력하기 위해 tqdm.auto에서 tqdm 가져오기
from sklearn.metrics import mean_squared_error      # 성능 평가 지표 함수 가져오기

import warnings                                     # 경고 제어를 위해 가져오기
warnings.filterwarnings(action='ignore')            # 경고 메세지 무시
 
from google.colab.patches import cv2_imshow         # google.colab.patches에서 cv2_imshow를 사용하기 위해 불러옴

from torchvision import transforms                  # 이미지 변환을 위해 transform을 불러옴

import PIL                                          # 이미지


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')   # cuda를 사용하기 위해 cuda로 설정

In [ ]:
from google.colab import drive              # 구글 drive 불러오기
drive.mount('/content/drive')               # 구글 코랩환경으로 실험 진행

Mounted at /content/drive


In [ ]:
!nvidia-smi                                 # 사용하는 프로세서 확인

Sat Dec 17 08:28:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip -qq "/content/drive/MyDrive/open.zip"    # 구글 드라이브에 있는 데이터 알집을 풀고 실험 진행

In [ ]:
CFG = {                                         # 동일한 parameter로 진행하기 위해 dictonary로 값 저장
    'WIDTH':48,
    'HEIGHT':72,
    'EPOCHS':2,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
}

## 데이터 전처리




In [ ]:
# case 1 
sem_1_80_paths = sorted(glob.glob('./simulation_data/SEM/Case_1/80/*.png'))
sem_1_81_paths = sorted(glob.glob('./simulation_data/SEM/Case_1/81/*.png'))
sem_1_82_paths = sorted(glob.glob('./simulation_data/SEM/Case_1/82/*.png'))
sem_1_83_paths = sorted(glob.glob('./simulation_data/SEM/Case_1/83/*.png'))
sem_1_84_paths = sorted(glob.glob('./simulation_data/SEM/Case_1/84/*.png'))

# case 2
sem_2_80_paths = sorted(glob.glob('./simulation_data/SEM/Case_2/80/*.png'))
sem_2_81_paths = sorted(glob.glob('./simulation_data/SEM/Case_2/81/*.png'))
sem_2_82_paths = sorted(glob.glob('./simulation_data/SEM/Case_2/82/*.png'))
sem_2_83_paths = sorted(glob.glob('./simulation_data/SEM/Case_2/83/*.png'))
sem_2_84_paths = sorted(glob.glob('./simulation_data/SEM/Case_2/84/*.png'))

# case 3
sem_3_80_paths = sorted(glob.glob('./simulation_data/SEM/Case_3/80/*.png'))
sem_3_81_paths = sorted(glob.glob('./simulation_data/SEM/Case_3/81/*.png'))
sem_3_82_paths = sorted(glob.glob('./simulation_data/SEM/Case_3/82/*.png'))
sem_3_83_paths = sorted(glob.glob('./simulation_data/SEM/Case_3/83/*.png'))
sem_3_84_paths = sorted(glob.glob('./simulation_data/SEM/Case_3/84/*.png'))

# case 4
sem_4_80_paths = sorted(glob.glob('./simulation_data/SEM/Case_4/80/*.png'))
sem_4_81_paths = sorted(glob.glob('./simulation_data/SEM/Case_4/81/*.png'))
sem_4_82_paths = sorted(glob.glob('./simulation_data/SEM/Case_4/82/*.png'))
sem_4_83_paths = sorted(glob.glob('./simulation_data/SEM/Case_4/83/*.png'))
sem_4_84_paths = sorted(glob.glob('./simulation_data/SEM/Case_4/84/*.png'))

# case 1
depth_1_80_paths = sorted(glob.glob('./simulation_data/Depth/Case_1/80/*.png') + glob.glob('./simulation_data/Depth/Case_1/80/*.png'))
depth_1_81_paths = sorted(glob.glob('./simulation_data/Depth/Case_1/81/*.png') + glob.glob('./simulation_data/Depth/Case_1/81/*.png'))
depth_1_82_paths = sorted(glob.glob('./simulation_data/Depth/Case_1/82/*.png') + glob.glob('./simulation_data/Depth/Case_1/82/*.png'))
depth_1_83_paths = sorted(glob.glob('./simulation_data/Depth/Case_1/83/*.png') + glob.glob('./simulation_data/Depth/Case_1/83/*.png'))
depth_1_84_paths = sorted(glob.glob('./simulation_data/Depth/Case_1/84/*.png') + glob.glob('./simulation_data/Depth/Case_1/84/*.png'))

# case 2
depth_2_80_paths = sorted(glob.glob('./simulation_data/Depth/Case_2/80/*.png') + glob.glob('./simulation_data/Depth/Case_2/80/*.png'))
depth_2_81_paths = sorted(glob.glob('./simulation_data/Depth/Case_2/81/*.png') + glob.glob('./simulation_data/Depth/Case_2/81/*.png'))
depth_2_82_paths = sorted(glob.glob('./simulation_data/Depth/Case_2/82/*.png') + glob.glob('./simulation_data/Depth/Case_2/82/*.png'))
depth_2_83_paths = sorted(glob.glob('./simulation_data/Depth/Case_2/83/*.png') + glob.glob('./simulation_data/Depth/Case_2/83/*.png'))
depth_2_84_paths = sorted(glob.glob('./simulation_data/Depth/Case_2/84/*.png') + glob.glob('./simulation_data/Depth/Case_2/84/*.png'))

# case 3
depth_3_80_paths = sorted(glob.glob('./simulation_data/Depth/Case_3/80/*.png') + glob.glob('./simulation_data/Depth/Case_3/80/*.png'))
depth_3_81_paths = sorted(glob.glob('./simulation_data/Depth/Case_3/81/*.png') + glob.glob('./simulation_data/Depth/Case_3/81/*.png'))
depth_3_82_paths = sorted(glob.glob('./simulation_data/Depth/Case_3/82/*.png') + glob.glob('./simulation_data/Depth/Case_3/82/*.png'))
depth_3_83_paths = sorted(glob.glob('./simulation_data/Depth/Case_3/83/*.png') + glob.glob('./simulation_data/Depth/Case_3/83/*.png'))
depth_3_84_paths = sorted(glob.glob('./simulation_data/Depth/Case_3/84/*.png') + glob.glob('./simulation_data/Depth/Case_3/84/*.png'))

# case 4
depth_4_80_paths = sorted(glob.glob('./simulation_data/Depth/Case_4/80/*.png') + glob.glob('./simulation_data/Depth/Case_4/80/*.png'))
depth_4_81_paths = sorted(glob.glob('./simulation_data/Depth/Case_4/81/*.png') + glob.glob('./simulation_data/Depth/Case_4/81/*.png'))
depth_4_82_paths = sorted(glob.glob('./simulation_data/Depth/Case_4/82/*.png') + glob.glob('./simulation_data/Depth/Case_4/82/*.png'))
depth_4_83_paths = sorted(glob.glob('./simulation_data/Depth/Case_4/83/*.png') + glob.glob('./simulation_data/Depth/Case_4/83/*.png'))
depth_4_84_paths = sorted(glob.glob('./simulation_data/Depth/Case_4/84/*.png') + glob.glob('./simulation_data/Depth/Case_4/84/*.png'))

In [ ]:
len_80 = len(sem_1_80_paths)
len_81 = len(sem_1_81_paths)
len_82 = len(sem_1_82_paths)
len_83 = len(sem_1_83_paths)
len_84 = len(sem_1_84_paths)

In [ ]:
ratio = 1
v_ratio = 0.2

In [ ]:
tran_1_80 = sem_1_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
tran_1_81 = sem_1_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
tran_1_82 = sem_1_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
tran_1_83 = sem_1_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
tran_1_84 = sem_1_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

tran_2_80 = sem_2_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
tran_2_81 = sem_2_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
tran_2_82 = sem_2_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
tran_2_83 = sem_2_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
tran_2_84 = sem_2_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

tran_3_80 = sem_3_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
tran_3_81 = sem_3_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
tran_3_82 = sem_3_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
tran_3_83 = sem_3_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
tran_3_84 = sem_3_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

tran_4_80 = sem_4_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
tran_4_81 = sem_4_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
tran_4_82 = sem_4_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
tran_4_83 = sem_4_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
tran_4_84 = sem_4_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

tran_1 = tran_1_80 + tran_1_81 + tran_1_82 + tran_1_83 + tran_1_84
tran_2 = tran_2_80 + tran_2_81 + tran_2_82 + tran_2_83 + tran_2_84
tran_3 = tran_3_80 + tran_3_81 + tran_3_82 + tran_3_83 + tran_3_84
tran_4 = tran_4_80 + tran_4_81 + tran_4_82 + tran_4_83 + tran_4_84

tot_tran_s = tran_1 + tran_2 + tran_3 + tran_4

In [ ]:
dep_tran_1_80 = depth_1_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
dep_tran_1_81 = depth_1_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
dep_tran_1_82 = depth_1_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
dep_tran_1_83 = depth_1_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
dep_tran_1_84 = depth_1_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

dep_tran_2_80 = depth_2_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
dep_tran_2_81 = depth_2_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
dep_tran_2_82 = depth_2_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
dep_tran_2_83 = depth_2_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
dep_tran_2_84 = depth_2_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

dep_tran_3_80 = depth_3_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
dep_tran_3_81 = depth_3_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
dep_tran_3_82 = depth_3_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
dep_tran_3_83 = depth_3_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
dep_tran_3_84 = depth_3_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

dep_tran_4_80 = depth_4_80_paths[int(len_80*v_ratio):int(len_80*ratio)]
dep_tran_4_81 = depth_4_81_paths[int(len_81*v_ratio):int(len_81*ratio)]
dep_tran_4_82 = depth_4_82_paths[int(len_82*v_ratio):int(len_82*ratio)]
dep_tran_4_83 = depth_4_83_paths[int(len_83*v_ratio):int(len_83*ratio)]
dep_tran_4_84 = depth_4_84_paths[int(len_84*v_ratio):int(len_84*ratio)]

dep_tran_1 = dep_tran_1_80 + dep_tran_1_81 + dep_tran_1_82 + dep_tran_1_83 + dep_tran_1_84
dep_tran_2 = dep_tran_2_80 + dep_tran_2_81 + dep_tran_2_82 + dep_tran_2_83 + dep_tran_2_84
dep_tran_3 = dep_tran_3_80 + dep_tran_3_81 + dep_tran_3_82 + dep_tran_3_83 + dep_tran_3_84
dep_tran_4 = dep_tran_4_80 + dep_tran_4_81 + dep_tran_4_82 + dep_tran_4_83 + dep_tran_4_84

tot_tran_d = dep_tran_1 + dep_tran_2 + dep_tran_3 + dep_tran_4

In [ ]:
val_1_80 = sem_1_80_paths[:int(len_80*v_ratio)]
val_1_81 = sem_1_81_paths[:int(len_81*v_ratio)]
val_1_82 = sem_1_82_paths[:int(len_82*v_ratio)]
val_1_83 = sem_1_83_paths[:int(len_83*v_ratio)]
val_1_84 = sem_1_84_paths[:int(len_84*v_ratio)]


val_2_80 = sem_2_80_paths[:int(len_80*v_ratio)]
val_2_81 = sem_2_81_paths[:int(len_81*v_ratio)]
val_2_82 = sem_2_82_paths[:int(len_82*v_ratio)]
val_2_83 = sem_2_83_paths[:int(len_83*v_ratio)]
val_2_84 = sem_2_84_paths[:int(len_84*v_ratio)]


val_3_80 = sem_3_80_paths[:int(len_80*v_ratio)]
val_3_81 = sem_3_81_paths[:int(len_81*v_ratio)]
val_3_82 = sem_3_82_paths[:int(len_82*v_ratio)]
val_3_83 = sem_3_83_paths[:int(len_83*v_ratio)]
val_3_84 = sem_3_84_paths[:int(len_84*v_ratio)]


val_4_80 = sem_4_80_paths[:int(len_80*v_ratio)]
val_4_81 = sem_4_81_paths[:int(len_81*v_ratio)]
val_4_82 = sem_4_82_paths[:int(len_82*v_ratio)]
val_4_83 = sem_4_83_paths[:int(len_83*v_ratio)]
val_4_84 = sem_4_84_paths[:int(len_84*v_ratio)]


val_1 = val_1_80 + val_1_81 + val_1_82 + val_1_83 + val_1_84
val_2 = val_2_80 + val_2_81 + val_2_82 + val_2_83 + val_2_84
val_3 = val_3_80 + val_3_81 + val_3_82 + val_3_83 + val_3_84
val_4 = val_4_80 + val_4_81 + val_4_82 + val_4_83 + val_4_84

val_s = val_1 + val_2 + val_3 + val_4

In [ ]:
dep_val_1_80 = depth_1_80_paths[:int(len_80*v_ratio)]
dep_val_1_81 = depth_1_81_paths[:int(len_81*v_ratio)]
dep_val_1_82 = depth_1_82_paths[:int(len_82*v_ratio)]
dep_val_1_83 = depth_1_83_paths[:int(len_83*v_ratio)]
dep_val_1_84 = depth_1_84_paths[:int(len_84*v_ratio)]
dep_val_1 = dep_val_1_80 + dep_val_1_81 + dep_val_1_82 + dep_val_1_83 + dep_val_1_84

dep_val_2_80 = depth_2_80_paths[:int(len_80*v_ratio)]
dep_val_2_81 = depth_2_81_paths[:int(len_81*v_ratio)]
dep_val_2_82 = depth_2_82_paths[:int(len_82*v_ratio)]
dep_val_2_83 = depth_2_83_paths[:int(len_83*v_ratio)]
dep_val_2_84 = depth_2_84_paths[:int(len_84*v_ratio)]
dep_val_2 = dep_val_2_80 + dep_val_2_81 + dep_val_2_82 + dep_val_2_83 + dep_val_2_84

dep_val_3_80 = depth_3_80_paths[:int(len_80*v_ratio)]
dep_val_3_81 = depth_3_81_paths[:int(len_81*v_ratio)]
dep_val_3_82 = depth_3_82_paths[:int(len_82*v_ratio)]
dep_val_3_83 = depth_3_83_paths[:int(len_83*v_ratio)]
dep_val_3_84 = depth_3_84_paths[:int(len_84*v_ratio)]
dep_val_3 = dep_val_3_80 + dep_val_3_81 + dep_val_3_82 + dep_val_3_83 + dep_val_3_84

dep_val_4_80 = depth_4_80_paths[:int(len_80*v_ratio)]
dep_val_4_81 = depth_4_81_paths[:int(len_81*v_ratio)]
dep_val_4_82 = depth_4_82_paths[:int(len_82*v_ratio)]
dep_val_4_83 = depth_4_83_paths[:int(len_83*v_ratio)]
dep_val_4_84 = depth_4_84_paths[:int(len_84*v_ratio)]
dep_val_4 = dep_val_4_80 + dep_val_4_81 + dep_val_4_82 + dep_val_4_83 + dep_val_4_84

val_dep = dep_val_1 + dep_val_2 + dep_val_3 + dep_val_4

In [ ]:
train_sem_paths = tot_tran_s
train_depth_paths = tot_tran_d

val_sem_paths = val_s
val_depth_paths = val_dep

In [ ]:
class TransformedDataset(Dataset):           
    def __init__(self, sem_path_list, depth_path_list, transform = None):
        self.sem_path_list = sem_path_list
        self.depth_path_list = depth_path_list
        self.transform = transform
        
    def __getitem__(self, index):
        sem_path = self.sem_path_list[index]
        sem_img = PIL.Image.open(sem_path).convert("L")
        if(self.transform):                                                 
            sem_img = self.transform(sem_img)
        sem_img = np.array(sem_img)
        sem_img = np.expand_dims(sem_img, axis=-1).transpose(2,0,1)
        sem_img = sem_img / 255.

        if self.depth_path_list is not None:
            depth_path = self.depth_path_list[index]
            depth_img = PIL.Image.open(depth_path).convert("L")
            if(self.transform):                                       
                depth_img = self.transform(depth_img)
            depth_img = np.array(depth_img)
            depth_img = np.expand_dims(depth_img, axis=-1).transpose(2,0,1)
            depth_img = depth_img / 255.

            return torch.Tensor(sem_img), torch.Tensor(depth_img)  
        
        else:
            img_name = sem_path.split('/')[-1]                                  
            
            return torch.Tensor(sem_img), img_name                     
          
            
    def __len__(self):
        return len(self.sem_path_list)

In [ ]:
jitterTransform08 = transforms.ColorJitter(brightness = [0.8,0.8])
jitterTransform09 = transforms.ColorJitter(brightness = [0.9,0.9])

t_jitter08 = TransformedDataset(train_sem_paths,train_depth_paths, transform = jitterTransform08)
t_jitter09 = TransformedDataset(train_sem_paths,train_depth_paths, transform = jitterTransform09)


In [ ]:
t_origindataset = TransformedDataset(train_sem_paths, train_depth_paths,None)

train_dataset = t_origindataset + t_jitter08 + t_jitter09 

In [ ]:
v_origindataset = TransformedDataset(val_sem_paths, val_depth_paths,None)

valid_dataset = v_origindataset

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle = True, num_workers = 6)
valid_loader = DataLoader(valid_dataset, batch_size = CFG['BATCH_SIZE'], shuffle = False, num_workers = 6)

In [ ]:
len(train_dataset), len(valid_dataset)

(415956, 34652)

In [ ]:
# train-SEM 데이터
sem_paths = sorted(glob.glob('/content/train/SEM/*/*/*.png'), key=lambda x : x[-14:-4])  \

depth_paths = sorted(glob.glob("/content/train_to_depth/*.png"))

sem_paths[:5], depth_paths[:5]

(['/content/train/SEM/Depth_140/site_00201/SEM_000001.png',
  '/content/train/SEM/Depth_140/site_00413/SEM_000002.png',
  '/content/train/SEM/Depth_140/site_00097/SEM_000005.png',
  '/content/train/SEM/Depth_140/site_00099/SEM_000006.png',
  '/content/train/SEM/Depth_140/site_00475/SEM_000008.png'],
 [])

In [ ]:
# train : val = 8 : 2 구분
add_train_sem_paths = sem_paths[:int(len(sem_paths)*0.8)]
add_val_sem_paths = sem_paths[int(len(sem_paths)*0.8):]

add_train_depth_paths = depth_paths[:int(len(sem_paths)*0.8)]
add_val_depth_paths = depth_paths[int(len(sem_paths)*0.8):]

In [ ]:
at_origindataset = TransformedDataset(add_train_sem_paths, add_train_depth_paths) 

add_train_dataset = at_origindataset
# add_train_dataset 생성

In [ ]:
av_origindataset = TransformedDataset(add_val_sem_paths,add_val_depth_paths) 

add_valid_dataset = av_origindataset
# add_valid_dataset 생성

In [ ]:
add_train_loader = DataLoader(add_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)

add_val_loader = DataLoader(add_valid_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

## Model Define

                   

In [ ]:
class NewModel(nn.Module):
    def __init__(self):
        super(NewModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1,64,3, stride=2, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,128,3, stride=2, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128,256,3,stride=2, padding = 1),
            nn.ReLU()
            )
        
        self.flatten = nn.Sequential(nn.Flatten(start_dim = 1), nn.Dropout(0.2))

        self.unflatten = nn.Unflatten(dim=1, unflattened_size = (256,9,6))
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256,128,3,stride=2,padding = 1,output_padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128,64,3, stride=2, padding = 1, output_padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64,1,3, stride=2, padding = 1, output_padding = 1)
        )
        
    def forward(self, x):
        x = self.encoder(x)

        x = self.flatten(x)

        x = self.unflatten(x)

        x = self.decoder(x)                                                    
        return x

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, epochs, device):  
    model.to(device)                                              
    criterion_img = nn.L1Loss().to(device)   
    best_score = 999999
    best_model = None
    best_hist = []
    for epoch in range(1, epochs + 1):                
        model.train()
       
        for sem, depth in tqdm(iter(train_loader)):                           
            sem = sem.float().to(device)                               
            depth = depth.float().to(device)
            
            optimizer.zero_grad()                                 
            model_pred_img = model(sem)                                    
            loss_img = criterion_img(model_pred_img, depth)               
            loss_img.backward()                                        
            optimizer.step()                                                    
            train_loss.append(loss_img.item())                            
            
        val_img_loss, val_img_rmse = validation(model, criterion_img,val_loader, device) 
        
        if best_score > val_img_rmse:                                
            best_score = val_img_rmse
            best_model = model
            best_hist = [epoch, np.mean(train_loss), val_img_rmse]
            best_rmse.append(best_score)
            
        if scheduler is not None:                                               
            scheduler.step()
    
    return best_model

In [ ]:
def validation(model, criterion_img, val_loader, device):                 
    model.eval()                                                     
    b_rmse_img = nn.MSELoss().to(device)                                      
    

    with torch.no_grad():                                                     
        for sem, depth in tqdm(iter(val_loader)):                             
            sem = sem.float().to(device)                        
            depth = depth.float().to(device)
            
            model_pred_img = model(sem)                                     
            loss_img = criterion_img(model_pred_img, depth)                  
            pred = (model_pred_img*255.).type(torch.int8).float()           
            true = (depth*255.).type(torch.int8).float()                      

            b_rmse_img = torch.sqrt(criterion_img(pred, true))      

            val_img_loss.append(loss_img.item())                            
            val_img_rmse.append(b_rmse_img.item())

            # Total
            total_loss = loss_img
            b_rmse_total = torch.sqrt((total_loss))

    return np.mean(val_img_loss), np.mean(val_img_rmse)

## Run

In [ ]:
model = NewModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: epoch)                                  

infer_model = train(model, optimizer, train_loader, valid_loader, scheduler, CFG['EPOCHS'], device)

In [ ]:
    df_loss=pd.Series(train_loss)
    print(df_loss)
    df_loss.plot()

    best=pd.Series(best_rmse)
    print(best)
    best.plot()

    val_loss=pd.Series(val_img_loss)
    print(val_loss[int(2165):])
    val_loss[int(2165):].plot()

    val_rmse=pd.Series(val_img_rmse)
    print(val_rmse)
    val_rmse[int(2165):].plot()

## Verify

In [ ]:
def train_1(model, optimizer, train_loader, val_loader, scheduler, device):    
    model.to(device)                                              
    criterion_img = nn.L1Loss().to(device)   
                                                                                         
    best_score = 999999
    best_model = None
    best_hist = []
    for epoch in range(1, CFG['EPOCHS']+1):                         
        model.eval()
        train_loss = []                                                      
       
        for sem, depth in tqdm(iter(train_loader)):                       
                                                                         
            sem = sem.float().to(device)                                
            depth = depth.float().to(device)
            optimizer.zero_grad()                                         
            
            model_pred_img = model(sem)                                    
            
            loss_img = criterion_img(model_pred_img, depth)                
            
            total_loss = loss_img

            total_loss.backward()

            optimizer.step()                                                    
            
            train_loss.append(total_loss.item())                            
            
        
        val_img_loss, val_img_rmse = validation(model, criterion_img,val_loader,device) 
    
        if best_score > val_img_rmse:                                     
            best_score = val_img_rmse
            best_model = model
            best_hist = [epoch, np.mean(train_loss), val_img_rmse]
            best_rmse.append(best_score)
        
        if scheduler is not None:                                               
            scheduler.step()
    
    return best_model

In [ ]:
def validation_1(model, criterion_img, val_loader,device):                  
    model.eval()                                                             
    b_rmse_img = nn.MSELoss().to(device)                               
    
    val_img_loss = []                                                      
    val_img_rmse = []


    with torch.no_grad():                                               
        for sem, depth in tqdm(iter(val_loader)):                         
            sem = sem.float().to(device)                                  
            depth = depth.float().to(device)
            
            model_pred_img = model(sem)                                     
            loss_img = criterion_img(model_pred_img, depth)                
            # image 
            pred = (model_pred_img*255.).type(torch.int8).float()        
            true = (depth*255.).type(torch.int8).float()                    

            b_rmse_img = torch.sqrt(criterion_img(pred, true))                

            val_img_loss.append(loss_img.item())                            
            val_img_rmse.append(b_rmse_img.item())

            # Total
    return np.mean(val_img_loss), np.mean(val_img_rmse)

In [ ]:
Model = infer_model
Model.eval()
optimizer = torch.optim.Adam(params = Model.parameters(), lr = 1e-3)            
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: epoch)

infer_model = train_1(Model, optimizer, add_train_loader, add_val_loader, scheduler,device)

In [ ]:
    df_loss=pd.Series(train_loss)
    print(df_loss)
    df_loss.plot()

    best=pd.Series(best_rmse)
    print(best)
    best.plot()

    val_loss=pd.Series(val_img_loss)
    print(val_loss[int(2165):])
    val_loss[int(2165):].plot()

    val_rmse=pd.Series(val_img_rmse)
    print(val_rmse)
    val_rmse[int(2165):].plot()